### import of important libraries

**Competition** : Kaggle at Ecole Centrale Lille  <br>
**Team**: Ayoub youssoufi, Yasser Zidani, Hatim Alouane 

In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

###x = train.iloc[:, 1:2].values
###y = train.iloc[:, 2:].values

import unicodedata
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re

import pandas as pd
import numpy as np
import re


from sklearn.metrics import roc_auc_score

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import string
import re
import nltk
from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader




from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

%matplotlib inline
%config InlineBackend.figure_format='retina'


### Import DataSets & Pre Processing: 

In [ ]:
#import X_trainn 
import pandas as pd
X_train = pd.read_csv("/content/X_trainn.csv", sep=';' , header= None, encoding='latin-1',names=["Id", "Caption"])
X_train=X_train[1:]

#import y_train
y_train = pd.read_csv("y_train.csv",sep=';', header=None,names=["Id","Catégorie1", "Catégorie2", "Catégorie3", "Catégorie4" ])
y_train=y_train[1:]

#import nonlabeled
nonlabeled = pd.read_csv("nonlabeled_data.csv",sep=';', header=None,names=["Id", "Caption"])
nonlabeled=nonlabeled[1:]

#we have investigated that y_train isn't of a type of int 
y_train.astype(int) 
y_train.iloc[:, 2:].values.astype(int)

#merge X_trainn and y_train so that we have a full dataset of inputs which are captions and 4 outputs= multi labels
train = pd.merge(X_train, y_train, on='Id')
#the same for Categories, Id we have found that it's not on int type
y_train["Catégorie1"].astype(int)
y_train["Catégorie2"].astype(int)
y_train["Catégorie3"].astype(int)
y_train["Catégorie4"].astype(int)
train["Id"] = train["Id"].astype(int)
train["Caption"] = train["Caption"].astype(str)

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

def clean_reponses(df):
  #initialise la liste des tokens 
  tokens = []
  
  #boucle sur chaque réponse
  for reponse in df:
    # retire les urls
    re_url = re.compile(r'https?://\S+|www\.\S+')
    reponse = re_url.sub('', reponse)
    # retire les ponctuations
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    reponse = re_punc.sub(' ', reponse)
    # convertit en minuscules
    reponse = reponse.lower()
    # tokenisation
    reponse = word_tokenize(reponse)
    # ajoute les tokens à la liste
    tokens.append(reponse)
    
  return tokens
rep_train = clean_reponses(train['Caption'])
rep_nonlabeled = clean_reponses(nonlabeled['Caption'])
rep_train = clean_reponses(train['Caption'])
rep_nonlabeled = clean_reponses(nonlabeled['Caption'])
rep_train = clean_reponses(train['Caption'].astype(str))
rep_nonlabeled = clean_reponses(nonlabeled['Caption'].astype(str))


In [ ]:
#Investigation of the length of the captions 
reponses_word_count = []

for i in train['Caption']:
  
  if isinstance(i, str):
        reponses_word_count.append(len(i.split()))
  else:
        reponses_word_count.append(0)

for i in nonlabeled['Caption']:
      reponses_word_count.append(len(i.split()))

length_df = pd.DataFrame({'Longueur des réponses':reponses_word_count})

length_df.hist(bins = 100, range=(0,100),figsize=(8,4))
plt.show()

In [ ]:
stop_words = set(stopwords.words('french'))

def clean_reponses(df):
  re_punc = re.compile('[%s]' % re.escape(string.punctuation))
  #retire la ponctuation
  tokens = [re_punc.sub(' ',reponse) for reponse in df]
  #convertit en minuscules
  tokens = [reponse.lower() for reponse in tokens]
  
  #split chaque réponse en liste de mots
  reponses = []
  reponses = [reponse.split() for reponse in tokens]
  
  #deleting stop_words
  for reponse in reponses:
    for word in reponse:
      if word in stop_words:
        del reponse[reponse.index(word)]
        
  #deleting words of length <= 2
  for reponse in reponses:
    for word in reponse:
      if len(word) <= 2:
        del reponse[reponse.index(word)]
  return reponses

  # Let's see the words the most used: 
rep_train = clean_reponses(train['Caption'])
rep_nonlabeled = clean_reponses(nonlabeled['Caption'])
l=[]
for reponse in rep_train:
  for word in reponse:
    l.append(word)

for reponse in rep_nonlabeled:
  for word in reponse:
    l.append(word)


def cleanHtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext


def cleanPunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned


def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent

def preprocess_text(text):
    # Normalize the text to remove any accented characters
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    # Replace any non-alphanumeric characters with a space
    text = re.sub(r'\W+', ' ', text)
    # Convert the text to lowercase
    text = text.lower()
    return text

stemmer = SnowballStemmer("french")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

# Function to clean text of any unwanted symbols and punctuation marks
def cleaning_text(text):
    text = text.lower()
    text = re.sub('r<.*?>', ' ', text)
    text = re.sub(r'#\w+', ' ', text)
    text = re.sub(r'@\w+', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    text = re.sub(r'http\S+', " ", text)

    text = text.split()
    stop_words = stopwords.words("english")
    text = " ".join([word for word in text if not word in stop_words])

    for punctuation in string.punctuation:
        text = text.replace(punctuation, "")

    return text

train['Caption'] = train['Caption'].apply(lambda x: cleaning_text(x))



train['Caption'] = train['Caption'].str.lower()
train['Caption'] = train['Caption'].apply(cleanHtml)
train['Caption'] = train['Caption'].apply(cleanPunc)
train['Caption'] = train['Caption'].apply(keepAlpha)
train['Caption'] = train['Caption'].apply(preprocess_text)
train['Caption'] = train['Caption'].apply(stemming)

rowSums = train.iloc[:,2:].sum(axis=1)
clean_comments_count = (rowSums==0).sum(axis=0)

print("Total number of captions = ",len(train))
print("Number of clean captions = ",clean_comments_count)
print("Number of captions with labels =",(len(train)-clean_comments_count))

In [ ]:
#Look for the balance in data
cat1_count = sum(train["Catégorie1"].astype(int))
cat2_count = sum(train["Catégorie2"].astype(int))
cat3_count = sum(train["Catégorie3"].astype(int))
cat4_count = sum(train["Catégorie4"].astype(int))

total_count = train.shape[0]

# Calculate the proportion of each category
cat1_proportion = cat1_count / total_count
cat2_proportion = cat2_count / total_count
cat3_proportion = cat3_count / total_count
cat4_proportion = cat4_count / total_count

print("Catégorie 1 proportion: {:.2f}".format(cat1_proportion))
print("Catégorie 2 proportion: {:.2f}".format(cat2_proportion))
print("Catégorie 3 proportion: {:.2f}".format(cat3_proportion))
print("Catégorie 4 proportion: {:.2f}".format(cat4_proportion))

LABEL_COLUMNS = train.columns.tolist()[2:]
train[LABEL_COLUMNS] = train[LABEL_COLUMNS].apply(pd.to_numeric, errors='coerce')
train[LABEL_COLUMNS].sum().sort_values().plot(kind="barh");


In [ ]:
#Let's stock the categories in a list that will be benfecial for us when we try to predict probabilities using softmax after
categories = list(train.columns.values)
categories = categories[2:]
print(categories)

### **1st Try**: using the most simple model after preprocessing the data, **logistic regression** with **TfidfVectorizer** on each category among the fourth
We will work on just the train dataset by dividing the train into trainset and testset, and we can see that for this multi label classification we did have an accuracy of about 80% for one of the four classes at least. 
Also, we started from the hypothesis of predicting labels for each category among the fourth categories, and that can be better than predicting simultaneously all the classes in the same time, because in the context of covid pandemic for a human being, we assumed that a human being is only interested in one category among the four in majority, so there is always a first important preoccupation of the person who's answer is in train["Caption"], and in the other hand: we are essentially training our model to learn specific patterns and features that are associated with each of those categories. This allows my model to make more accurate predictions for each of the categories. So, we assumed that the relationships between the categories are weak or the correlations between them are not well strong



In [ ]:
from sklearn.model_selection import train_test_split

trainset, testset = train_test_split(train, random_state=42, test_size=0.30, shuffle=True)
train_text = trainset['Caption']
test_text = testset['Caption']
train_text = trainset['Caption'].values.astype('U')
test_text = testset['Caption'].values.astype('U')

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2')
vectorizer.fit(train_text)
vectorizer.fit(test_text)

xx_train = vectorizer.transform(train_text)
yy_train = trainset.drop(labels = ['Id','Caption'], axis=1)


xx_test = vectorizer.transform(test_text)
yy_test = testset.drop(labels = ['Id','Caption'], axis=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
#printmd('**bo

# Using pipeline for applying logistic regression and one vs rest classifier
LogReg_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),
            ])

for category in categories:
    printmd('**Processing {} captions...**'.format(category))
    
    # Training logistic regression model on train data
    LogReg_pipeline.fit(xx_train, trainset[category])
    
    # calculating test accuracy
    prediction = LogReg_pipeline.predict(xx_test)
    print('Test accuracy is {}'.format(accuracy_score(testset[category], prediction)))
    print("\n") 

### 2nd Try:  **EBert Model**

In [ ]:
!pip install -q transformers
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
from torch.utils.data import Dataset
max_len = 32
# Class PrepDataset to format the dataset to feed the BERT model
class PrepDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe['Caption']
        self.targets = self.data.Label
        self.max_len = max_len
    def __len__(self):
        return len(self.text)
    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
train['Label']=train[['Catégorie1', 'Catégorie2', 'Catégorie3', 'Catégorie4']].values.tolist()
train = train.drop(columns = ['Catégorie1', 'Catégorie2', 'Catégorie3', 'Catégorie4'])
train

In [ ]:
#Split 8:2 ratio train-validation
from torch.utils.data import DataLoader

train_size = 0.8
train_dataset = train.sample(frac=train_size,random_state=200)
valid_dataset = train.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)
training_set = PrepDataset(train_dataset, tokenizer, max_len)
validation_set = PrepDataset(valid_dataset, tokenizer, max_len)
training_loader = DataLoader(training_set, batch_size=30)
validation_loader = DataLoader(validation_set, batch_size=30)

In [ ]:
from transformers import BertModel
class BERT(torch.nn.Module):
    def __init__(self):
        super(BERT, self).__init__()
        self.l1 = BertModel.from_pretrained('bert-base-uncased', return_dict=False)
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 4)
    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

In [ ]:
import tensorflow as tf

# Verify the use of GPU in google colab
device_name = tf.test.gpu_device_name()
# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


In [ ]:
# Training
epochs = 7
learning_rate = 2e-10
model = BERT()
model.to(device)
len_trainloader = len(training_loader)
loss_f = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
train_loss = 0
valid_loss = 0
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)
for epoch in range(epochs):
  avg_loss = 0
  items = 0
  for batch_idx, data in enumerate(training_loader):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)
        outputs = model(ids, mask, token_type_ids)
        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
  print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')

In [ ]:
val_targets=[]
val_outputs=[]
valid_loss=0
from sklearn import metrics

model.eval()
with torch.no_grad():
    for batch_idx, data in enumerate(validation_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)
        outputs = model(ids, mask, token_type_ids)
        loss = loss_fn(outputs, targets)
        valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
        val_targets.extend(targets.cpu().detach().numpy().tolist())
        val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())



val_preds = (np.array(val_outputs) > 0.3).astype(int)
accuracy = metrics.accuracy_score(val_targets, val_preds)
f1 = metrics.f1_score(val_targets, val_preds, average='micro')

# **Use of Camembert, the best approach (highest score)**

Predicting labels for each category among the fourth (or only the first three) can be better than doing a prediction on all labels in the same time because:

Better accuracy: The first three categories are well-defined and mutually exclusive, so predicting labels for each category would give us a better accuracy than predicting all the labels at the same time or on the fourth label which is a combination of all labels.

Better insights: Predicting labels for each category can give us better insights into the specific **(very Real, most important)** concerns of the respondents. This can help in developing better strategies for addressing these concerns.

Better decision-making: Predicting labels for each category can help decision-makers to prioritize their actions based on the most pressing concerns. This can help in making better decisions on how to allocate resources to address the concerns of the respondents.

Therefore, predicting labels for each category can be a more efficient and effective way to analyze the dataset and understand the concerns of the respondentS;

It's worth to say, that we trained the model separately in terms of each label based on the assumption that we developed before, and we were playing on hyperparameters of our model in each category so that we achieve the highest scores, and in the same time we didnt use non labeled data but we keeped an eye on the difference between train loss and validation loss so that we are sure that the model isn't overfitting, and then we concatenate the prediction of probabilities for the 4 categories to have our submission file 

#### Category_1

In [ ]:
# Reimporte the X_train, y_train because we had to restart kernel many times to update tokenizer (else it outout a nontype error)
X_train = pd.read_excel("X_train1.xlsx",header=None,names=["Id", "Caption"])[1:]
X_train["Id"] = X_train["Id"].astype(int)
y_train = pd.read_csv("y_train.csv",sep=';',header=None,names=["Id","category_1","category_2","category_3","category_4"])[1:]
y_train=y_train.astype(int)

In [ ]:
!pip install -q transformers

In [ ]:
# Importing standard libraries for every machine/deep learning pipeline
import pandas as pd
import torch
from tqdm import tqdm, trange
import numpy as np


# Importing specific libraries for data prerpcessing, model archtecture choice, training and evaluation
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler 
from transformers import CamembertTokenizer, CamembertForSequenceClassification
from transformers import AdamW
# import torch.optim as optim
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# import seaborn as sns

In [ ]:
epochs = 4
MAX_LEN = 64
batch_size = 32
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
!pip install datsets transformers[sentencepiece]
!pip install sentencepiece

In [ ]:
tokenizer = CamembertTokenizer.from_pretrained('camembert-base',do_lower_case=True)

In [ ]:
text = X_train['Caption'].to_list()
labels = y_train['category_1'].to_list()

for i in range(len(text)):
  if pd.isna(text[i]):
    print(i)

In [ ]:
text[88]='aucune'
text[240]="#NOM?"
text[333]="#NOM?"

In [ ]:
input_ids  = [tokenizer.encode(sent,add_special_tokens=True,max_length=MAX_LEN) for sent in text]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]  
    attention_masks.append(seq_mask)

train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks = train_test_split(input_ids, labels, attention_masks,random_state=42, test_size=0.1)

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [ ]:
model = CamembertForSequenceClassification.from_pretrained("camembert-base", num_labels=2)
model.to(device) 

In [ ]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, eps=10e-8)

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
train_loss_set = []

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):  
    # Tracking variables for training
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
  
    # Train the model
    model.train()
    for step, batch in enumerate(train_dataloader):
        # Add batch to device CPU or GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()
        # Forward pass
        outputs = model(b_input_ids,token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        # Get loss value
        loss = outputs[0]
        # Add it to train loss list
        train_loss_set.append(loss.item())    
        # Backward pass
        loss.backward()
        # Update parameters and take a step using the computed gradient
        optimizer.step()
    
        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    print("Train loss: {}".format(tr_loss/nb_tr_steps))

    # Tracking variables for validation
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    # Validation of the model
    model.eval()
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        # Add batch to device CPU or GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Telling the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions
            outputs =  model(b_input_ids,token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            loss, logits = outputs[:2]
    
        # Move logits and labels to CPU if GPU is used
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

In [ ]:
## Call X_test:
X_test = pd.read_excel("X_test11.xlsx",header=None,names=["Id", "Caption"])[1:]
X_test["Id"] = X_test["Id"].astype(int)



In [ ]:
texttest = X_test['Caption'].to_list()



In [ ]:
for i in range(len(texttest)):
  if pd.isna(texttest[i]):
    print(i)

In [ ]:


texttest[73]="#NOM?"
texttest[122]="#NOM?"


In [ ]:
# Encode the comments
tokenized_comments_ids = [tokenizer.encode(comment,add_special_tokens=True,max_length=MAX_LEN) for comment in texttest]
# Pad the resulted encoded comments
tokenized_comments_ids = pad_sequences(tokenized_comments_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create attention masks 
attention_masks = []
for seq in tokenized_comments_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

prediction_inputs = torch.tensor(tokenized_comments_ids)
prediction_masks = torch.tensor(attention_masks)

In [ ]:
# Apply the finetuned model (Camembert)
flat_pred = []
with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs =  model(prediction_inputs.to(device),token_type_ids=None, attention_mask=prediction_masks.to(device))
    logits = outputs[0]
    logits = logits.detach().cpu().numpy() 
    flat_pred.extend(np.argmax(logits, axis=1).flatten())

In [ ]:
flat_pred=pd.DataFrame(flat_pred)
flat_pred

In [ ]:
from google.colab import files
flat_pred.to_csv('cat1.csv',index=False)
files.download('cat1.csv')

#### Category2: 

In [ ]:
text = X_train['Caption'].to_list()
labels = y_train['category_2'].to_list()

In [ ]:
for i in range(len(text)):
  if pd.isna(text[i]):
    print(i)

In [ ]:
text[88]='aucune'
text[240]="#NOM?"
text[333]="#NOM?"

In [ ]:
epochs = 9
MAX_LEN = 128
batch_size = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
input_ids  = [tokenizer.encode(sent,add_special_tokens=True,max_length=MAX_LEN) for sent in text]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]  
    attention_masks.append(seq_mask)

train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks = train_test_split(input_ids, labels, attention_masks,random_state=42, test_size=0.1)

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

model = CamembertForSequenceClassification.from_pretrained("camembert-base", num_labels=2)
model.to(device) 

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=2e-6, eps=10e-8)

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
train_loss_set = []

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):  
    # Tracking variables for training
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
  
    # Train the model
    model.train()
    for step, batch in enumerate(train_dataloader):
        # Add batch to device CPU or GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()
        # Forward pass
        outputs = model(b_input_ids,token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        # Get loss value
        loss = outputs[0]
        # Add it to train loss list
        train_loss_set.append(loss.item())    
        # Backward pass
        loss.backward()
        # Update parameters and take a step using the computed gradient
        optimizer.step()
    
        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    print("Train loss: {}".format(tr_loss/nb_tr_steps))

    # Tracking variables for validation
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    # Validation of the model
    model.eval()
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        # Add batch to device CPU or GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Telling the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions
            outputs =  model(b_input_ids,token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            loss, logits = outputs[:2]
    
        # Move logits and labels to CPU if GPU is used
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

In [ ]:
X_test = pd.read_excel("X_test11.xlsx",header=None,names=["Id", "Caption"])[1:]
X_test["Id"] = X_test["Id"].astype(int)
texttest = X_test['Caption'].to_list()

In [ ]:
for i in range(len(texttest)):
  if pd.isna(texttest[i]):
    print(i)

In [ ]:
texttest[73]='#NOM?'
texttest[122]='#NOM?'

In [ ]:
tokenized_comments_ids = [tokenizer.encode(comment,add_special_tokens=True,max_length=MAX_LEN) for comment in texttest]
# Pad the resulted encoded comments
tokenized_comments_ids = pad_sequences(tokenized_comments_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create attention masks 
attention_masks = []
for seq in tokenized_comments_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

prediction_inputs = torch.tensor(tokenized_comments_ids)
prediction_masks = torch.tensor(attention_masks)

In [ ]:
flat_pred = []
with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs =  model(prediction_inputs.to(device),token_type_ids=None, attention_mask=prediction_masks.to(device))
    logits = outputs[0]
    logits = logits.detach().cpu().numpy() 
    flat_pred.extend(np.argmax(logits, axis=1).flatten())

In [ ]:
cat3=np.array([1/(1+np.exp(-logits[i][1])) for i in range(len(logits))])
cat2=np.array([1/(1+np.exp(-logits[i][1])) for i in range(len(logits))])
cat1=np.array([1/(1+np.exp(-logits[i][1])) for i in range(len(logits))])
cat4=np.array([1/(1+np.exp(-logits[i][1])) for i in range(len(logits))])

In [ ]:
import pandas as pd
new = pd.read_csv("/content/newprob2.csv",sep=',',header=None,names=["Id","category_1","category_2","category_3","category_4"])[1:]
new=new.astype(float)
new

In [ ]:
for i in range(157):
  new.iloc[i][2]=cat2[i]

In [ ]:
new['Id']=new['Id'].astype(int)
new

In [ ]:
nb_new1=sum(new["category_1"].astype(int))
nb_new2=sum(new["category_2"].astype(int))
nb_new3=sum(new["category_3"].astype(int))
nb_new4=sum(new["category_4"].astype(int))

In [ ]:
y_train = pd.read_csv("y_train.csv",sep=';',header=None,names=["Id","category_1","category_2","category_3","category_4"])[1:]
y_train = y_train.astype(int)
y_train

In [ ]:
nb_cat1=sum(y_train["category_1"].astype(int))
nb_cat2=sum(y_train["category_2"].astype(int))
nb_cat3=sum(y_train["category_3"].astype(int))
nb_cat4=sum(y_train["category_4"].astype(int))

#### Interpretation: 

In [ ]:
import pandas as pd
new = pd.read_csv("newprob2.csv",sep=',',header=None,names=["Id","category_1","category_2","category_3","category_4"])[1:]
new=new.astype(float)
new

In [ ]:
prop4=0
for i in range(len(new)):
    if new.iloc[i][4]>0.9:
        prop4=prop4+1
prop4/157

In [ ]:
prop3=0
for i in range(len(new)):
    if new.iloc[i][3]>0.9:
        prop3=prop3+1
prop3/157

In [ ]:
prop1=0
for i in range(len(new)):
    if new.iloc[i][1]>0.9:
        prop1=prop1+1
prop1/157

In [ ]:
prop2=0
for i in range(len(new)):
    if new.iloc[i][2]>0.7:
        prop2=prop2+1
prop2/157